In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
class modeling:
  !pip install catboost
  # train과 test로 나누기
  def train_test_split(data = None):
    train = data[data['date'] <= 201712]
    test = data[data['date'] > 201712]

    X_train, y_train = train.drop('target', axis = 1), train['target']
    X_test, y_test = test.drop('target', axis = 1), test['target']

    return X_train, X_test, y_train, y_test

  # linear regression 돌리기
  def lr():
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    
    return model

  # xgboost 돌리기
  def xgb():
    from xgboost import XGBRegressor
    model = XGBRegressor(random_state = 42,tree_method = 'gpu_hist')

    return model

  # lightgbm
  def lgb():
    from lightgbm import LGBMRegressor
    model = LGBMRegressor(random_state = 42)

    return model

  # catboost
  def cat():
    from catboost import CatBoostRegressor
    model = CatBoostRegressor(random_state = 42,task_type = 'GPU')

    return model

  # ridge
  def ridge():
    from sklearn.linear_model import Ridge
    model = Ridge(random_state = 42)
    
    return model
  
  # lasso
  def lasso():
    from sklearn.linear_model import Lasso
    model = Lasso()

    return model

  # adaboost
  def ada():
    from sklearn.ensemble import AdaBoostRegressor
    model = AdaBoostRegressor(random_state = 42)

    return model

  # randomforest
  def rfg():
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(random_state = 42)

    return model

  def rmse(y_test, y_pred):
    from sklearn.metrics import mean_squared_error
    RMSE = mean_squared_error(y_test, y_pred)**0.5

    return RMSE


# 모델 전부 한번씩 돌려보기
class class_all_model:
  # 각 모델을 돌리기 위해서 dict로 쌓아준다.
  def models():
    models = {'lr':modeling.lr(), 'lgb':modeling.lgb(), 'xgb':modeling.xgb(), 'lasso':modeling.lasso(),'ridge':modeling.ridge(),'cat':modeling.cat(),'ada':modeling.ada(), 'rfg':modeling.rfg()}
    return models
  
  # 모델을 하나씩 돌려서 keys = 모델명, value = 모델의 rmse를 쌓아준다.
  def run(data = None):
    models = class_all_model.models()
    final_models = {}
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    for i, j in models.items():
      model = j
      model.fit(X_train, y_train)
      y_pred = model.predict(X_test)

      final_models[i] = modeling.rmse(y_test, y_pred)
    return final_models

# stacking ensemble
class class_stack:
  def stack(data = None):
    !pip install vecstack
    from sklearn.metrics import mean_squared_error
    from vecstack import stacking
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    models = [modeling.lgb(), modeling.cat(), modeling.lr(), modeling.xgb(), modeling.lasso(), modeling.ada(), modeling.rfg(),modeling.ridge()]
    # stacking을 사용하자
    S_train, S_test = stacking(models, X_train, y_train, X_test, regression = True, metric = ['acc'], n_folds = 3)
    model = modeling.lr()
    model.fit(S_train, y_train)
    y_pred = model.predict(S_test)

    return modeling.rmse(y_test, y_pred)


class class_voting:
  def collect_model():
    models= [('xgb',modeling.xgb()),('lgb',modeling.lgb()),('rfg',modeling.rfg()),('ada',modeling.ada()),('lr',modeling.lr()),('ridge',modeling.ridge()),('lasso',modeling.lasso()),('cat',modeling.cat())]
    return models

  def voting(data = None):
    from sklearn.ensemble import VotingRegressor
    from sklearn.metrics import mean_squared_error
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    model = VotingRegressor(class_voting.collect_model())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return modeling.rmse(y_test, y_pred)

class final_model:
  def run(data = None):
    final_rmse = class_all_model.run(data = data)
    final_rmse['stack'] = class_stack.stack(data =data)
    final_rmse['voting'] = class_voting.voting(data =data)

    return final_rmse
    print('------ 끝 --------------------------------------------------')

In [20]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/sd솔데스크/팀 프로젝트/덕수궁_겨울.csv')
data = data.rename(columns = {'덕수궁':'target'})
data.head()

,date,month,최고 기온(°C),최소 상대습도(%),평균 기온(°C),평균 상대습도(%),일강수량(mm),평균 풍속(m/s),합계 일조 시간(hr),0.5m 지중온도(°C),최대 풍속(m/s),합계 일사량(MJ/m2),target,season,겨울철 체감온도,불쾌지수
0,201101,1,-3.406452,3.587097,-7.183871,4.004220,0.747872,1.334152,2.085472,1.541935,1.886092,2.393457,92135,4,23.985270,39.623680
1,201102,2,5.864286,3.496508,1.221429,4.029425,2.370244,1.267953,1.938742,1.025000,1.830866,2.470458,132920,4,33.261087,46.761874
2,201112,12,2.851613,3.486684,-0.870968,3.940391,0.464782,1.285404,1.989199,5.529032,1.815140,2.074713,52140,4,30.955365,44.997064
3,201201,1,1.338710,3.451522,-2.787097,3.909180,0.443617,1.261937,1.966435,1.680645,1.847184,2.147628,40041,4,28.764408,43.375576
4,201202,2,2.968966,3.203587,-2.044828,3.788257,0.125163,1.352096,2.169645,0.968966,1.893320,2.502707,36900,4,29.766218,44.025280


In [ ]:
# class를 이용해서 모든 모델을 돌려준다.
final_rmse = final_model.run(data =data)

In [22]:
print(final_rmse)

{'lr': 59281.91837959518, 'lgb': 43249.31219833734, 'xgb': 46546.852529673466, 'lasso': 34340.14192086436, 'ridge': 54918.9608005728, 'cat': 42523.57605168319, 'ada': 42129.24915240994, 'rfg': 42323.81143020616, 'stack': 41544.55694923496, 'voting': 43017.37874952668}
